# 03 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to AI Platform Prediction, using Cloud Build.
The CI/CD rountine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in AI Platform Prediction if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.


## Setup

In [1]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

In [2]:
PROJECT = 'ksalama-cloudml'
REGION = 'us-central1'
MODEL_DISPLAY_NAME = 'chicago_taxi_tips_classifier_v1'
ENDPOINT_DISPLAY_NAME = 'chicago_taxi_tips_classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

REPO_URL = "https://github.com/ksalama/ucaip-labs.git"
BRANCH = "simplified"

## Build CI/CD  Container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
!echo $CICD_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

## Run CI/CD Steps locally

In [3]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

In [ ]:
!py.test src/tests/model_deployment_tests.py::test_model_artifact

### Run create endpoint

In [ ]:
!python build/deploy-model.py \
    --mode=endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

### Run deploy model

In [ ]:
!python build/deploy-model.py \
    --mode=model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

### Run model endpoint testing

In [ ]:
!py.test src/tests/model_deployment_tests.py::test_model_endpoint

## Execute the Model Deployment CI/CD rountine to Cloud Build

In [4]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

_REPO_URL=https://github.com/ksalama/ucaip-labs.git,_BRANCH=simplified,_CICD_IMAGE_URI=gcr.io/ksalama-cloudml/cicd:latest,_PROJECT=ksalama-cloudml,_REGION=us-central1,_MODEL_DISPLAY_NAME=chicago_taxi_tips_classifier_v1,_ENDPOINT_DISPLAY_NAME=chicago_taxi_tips_classifier,


In [5]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m

Created [https://cloudbuild.googleapis.com/v1/projects/ksalama-cloudml/locations/global/builds/2235b92b-92e2-4549-b3ea-ec1bb487fdd3].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/2235b92b-92e2-4549-b3ea-ec1bb487fdd3?project=900786220115].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "2235b92b-92e2-4549-b3ea-ec1bb487fdd3"

FETCHSOURCE
BUILD
Starting Step #0 - "Clone Repository"
Step #0 - "Clone Repository": Already have image (with digest): gcr.io/cloud-builders/git
Step #0 - "Clone Repository": Cloning into 'ucaip-labs'...
Step #0 - "Clone Repository": POST git-upload-pack (377 bytes)
Step #0 - "Clone Repository": POST git-upload-pack (194 bytes)
Finished Step #0 - "Clone Repository"
Starting Step #1 - "Test Model Artifact"
Step #1 - "Test Model Artifact": Pulling image: gcr.io/ksalama-cloudml/cicd:latest
Step #1 - "Test Model Artifact": latest: Pulling from ksalama-cloudml/cicd
Step #1 - "Test Model Artif